<a href="https://colab.research.google.com/github/Nell87/drivendata_richter/blob/main/script/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Model (Random Forest)
This first model will be used as comparison for the performance of next ones. 

### **INCLUDES**


In [ ]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3
!pip install pretty-confusion-matrix

In [15]:
####    0. INCLUDES  _______________________________________ #### 
#Loading Libraries:# 
import pandas as pd
import time   #  provides many ways of representing time in code

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from pretty_confusion_matrix import pp_matrix_from_data
from sklearn.metrics import f1_score   # the score used in the competition

####   1. READING TRAIN AND TEST DATA _______________________________________ #### 
train_values= data = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_values.csv",index_col='building_id')
train_labels = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_labels.csv",index_col='building_id')
test = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/test_values.csv",index_col='building_id')

### **PREPROCESSING**
Let's dummify and split in train/test

In [16]:
#### 2.  PREPROCESSING DATA     _______________________________________ #### 
# Dummify
train_values = pd.get_dummies(train_values, drop_first = True)

# Split in train/test
x_train, x_test, y_train, y_test = train_test_split(train_values, train_labels, test_size = 0.2, random_state = 42)

### **MODELING**
Let's train the baseline model: Random Forest. I'll obtain the best parameters through GridSearch using my own computer to speed up the process.

In [17]:
#### 3.  MODELING     _______________________________________ #### 
# Create the RF object
# rf_clf = RandomForestClassifier(random_state=314) 

# Best parameters (auto deprecated???) # 1141 seg (19 m)
#param_grid = { 
#     'n_estimators': [100, 200],
#    'max_features': ['none', 'auto', 'sqrt', 'log2'],
#    'max_depth' : [1,10],
#    'min_samples_leaf' : [10,20]
#}

#start_time = time.time()
#rf_clf_GS = GridSearchCV(rf_clf, param_grid, cv=5)
#rf_clf_GS.fit(x_train, y_train)
#dt_time_fit = time.time() - start_time
#rf_clf_GS.best_params_

# Train using the best parameters  # 12 seg
rf_clf_1 = RandomForestClassifier(random_state=314, n_estimators = 100,
                                  max_features = 'auto', max_depth = 10,
                                  min_samples_leaf = 20)
                                  
                                  
start_time = time.time()
rf_clf_1 = rf_clf_1.fit(x_train, y_train)  
rf_clf_1_time_fit = time.time() - start_time       

<ipython-input-17-1a1459b8b8a8>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf_1 = rf_clf_1.fit(x_train, y_train)


### **PREDICTIONS**

In [18]:
#### 4.  PREDICTIONS     _______________________________________ #### 

#Predictions to check # 0.631
pred_rf_clf_1 = rf_clf_1.predict(x_test)
f1_score(y_test,pred_rf_clf_1, average='micro')

# confusion matrix
confusion_matrix(y_test,pred_rf_clf_1)
pp_matrix_from_data(y_test, pred_rf_clf_1)


/usr/local/lib/python3.8/dist-packages/pretty_confusion_matrix/pretty_confusion_matrix.py:200: MatplotlibDeprecationWarning: 
The tick1On function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use Tick.tick1line.set_visible instead.
  t.tick1On = False
/usr/local/lib/python3.8/dist-packages/pretty_confusion_matrix/pretty_confusion_matrix.py:201: MatplotlibDeprecationWarning: 
The tick2On function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use Tick.tick2line.set_visible instead.
  t.tick2On = False
/usr/local/lib/python3.8/dist-packages/pretty_confusion_matrix/pretty_confusion_matrix.py:203: MatplotlibDeprecationWarning: 
The tick1On function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use Tick.tick1line.set_visible instead.
  t.tick1On = False
/usr/local/lib/python3.8/dist-packages/pretty_confusion_matrix/pretty_confusion_matrix.py:204: MatplotlibDeprecationWarning: 
The tick2On function was deprecated in Matplotlib 3.1 and will be remo

ImportError: ignored

<Figure size 648x648 with 1 Axes>

In [ ]:
# Predictions to send # 0.6312 on competition (1469 / 5974)
test = pd.get_dummies(test, drop_first = True)
pred_rf_clf_1_final = rf_clf_1.predict(test)

my_submission = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/submission_format.csv",
                            index_col='building_id')
                            
my_submission = pd.DataFrame(data=pred_rf_clf_1_final,
                             columns=my_submission.columns,
                             index=my_submission.index)

my_submission.head()
# my_submission.to_csv('../data/submission_rf_clf_1.csv')